# DS-SF-27 | Unit Project 3: Basic Machine Learning Modeling | Answer Key

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.

In [1]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)
pd.set_option('display.notebook_repr_html', True)

import statsmodels.formula.api as smf

from sklearn import linear_model

In [2]:
df = pd.read_csv(os.path.join('..', '..', 'dataset', 'ucla-admissions.csv'))
df.dropna(inplace = True)

df

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0
...,...,...,...,...
395,0,620.0,4.00,2.0
396,0,560.0,3.04,3.0
397,0,460.0,2.63,2.0
398,0,700.0,3.65,2.0


## Part A.  Frequency Table

> ### Question 1.  Create a frequency table for `prestige` and whether an applicant was admitted.

In [3]:
pd.crosstab(df.admit, df.prestige)

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


## Part B.  Variable Transformations

> ### Question 2.  Create a one-hot encoding for `prestige`.

In [4]:
prestige_df = pd.get_dummies(df.prestige, prefix = 'prestige')

prestige_df.rename(columns = {'prestige_1.0': 'prestige_1',
                              'prestige_2.0': 'prestige_2',
                              'prestige_3.0': 'prestige_3',
                              'prestige_4.0': 'prestige_4'}, inplace = True)

prestige_df.head()

,prestige_1,prestige_2,prestige_3,prestige_4
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0


> ### Question 3.  How many of these binary variables do we need for modeling?

Answer: All but one, so: $4 - 1 = 3$.

> ### Question 4.  Why are we doing this?

Answer: This is done to prevent multicollinearity or the "dummy" variable trap caused by including a dummy variable for every single category.

> ### Question 5.  Add all these binary variables in the dataset and remove the now redundant `prestige` feature.

In [5]:
df = df[ ['admit', 'gre', 'gpa'] ].join(prestige_df)

df.head()

,admit,gre,gpa,prestige_1,prestige_2,prestige_3,prestige_4
0,0,380.0,3.61,0.0,0.0,1.0,0.0
1,1,660.0,3.67,0.0,0.0,1.0,0.0
2,1,800.0,4.00,1.0,0.0,0.0,0.0
3,1,640.0,3.19,0.0,0.0,0.0,1.0
4,0,520.0,2.93,0.0,0.0,0.0,1.0


## Part C.  Hand calculating odds ratios

Let's develop our intuition about expected outcomes by hand calculating odds ratios.

> ### Question 6.  Create a frequency table for `prestige = 1` and whether an applicant was admitted.

In [6]:
pd.crosstab(df.admit, df.prestige_1)

prestige_1,0.0,1.0
admit,,
0,243,28
1,93,33


> ### Question 7.  Use the frequency table above to calculate the odds of being admitted to graduate school for applicants that attended the most prestigious undergraduate schools.

In [7]:
odds_admitted_tier_1 = 33. / 28

odds_admitted_tier_1

1.1785714285714286

> ### Question 8.  Now calculate the odds of admission for undergraduates who did not attend a #1 ranked college.

In [8]:
odds_admitted_NOT_tier_1 = 93. / 243

odds_admitted_NOT_tier_1

0.38271604938271603

> ### Question 9.  Finally, what's the odds ratio?

In [9]:
odds_ratio = odds_admitted_tier_1 / odds_admitted_NOT_tier_1

odds_ratio

3.079493087557604

> ### Question 10.  Write this finding in a sentence.

Answer:

1. For students who attended a top ranked college, the odds of being admitted into graduate school are 3 times the odds of students who did not OR,
2. The odds of being admitted to graduate school are 2 times higher for students who attended a top ranked college compared to students who did not.

> ### Question 11.  Use the frequency table above to calculate the odds of being admitted to graduate school for applicants that attended the least prestigious undergraduate schools.  Then calculate their odds ratio of being admitted to UCLA.  Finally, write this finding in a sentence.

In [10]:
pd.crosstab(df.admit, df.prestige_4)

prestige_4,0.0,1.0
admit,,
0,216,55
1,114,12


In [11]:
odds_admitted_tier_4 = 12. / 55
print 'odds_admitted_tier_4 =', odds_admitted_tier_4

odds_admitted_NOT_tier_4 = 114. / 216
print 'odds_admitted_NOT_tier_4 = ', odds_admitted_NOT_tier_4

odds_ratio = odds_admitted_tier_4 / odds_admitted_NOT_tier_4
print 'odds_ratio = ', odds_ratio

odds_admitted_tier_4 = 0.218181818182
odds_admitted_NOT_tier_4 =  0.527777777778
odds_ratio =  0.413397129187


Answer:

1. For students who attended a lowest tier college, the odds of being admitted into graduate school are .4 times the odds of students who did not OR,
2. The odds of being admitted to graduate school are 60% lower for students who attended the lowest tier college compared to students who did not.

## Part C. Analysis using `statsmodels`

> ### Question 12.  Fit a logistic regression model predicting admission into UCLA using `gre`, `gpa`, and the `prestige` of the undergraduate schools.  Use the highest prestige undergraduate schools as your reference point.

In [12]:
model = smf.logit(formula = 'admit ~ gre + gpa + prestige_2 + prestige_3 + prestige_4', data = df).fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


> ### Question 13.  Print the model's summary results.

In [13]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                                   Pseudo R-squ.:                 0.08166
Time:                                   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -3.8769      1.142     -3.393      0.001        -6.116    -1.638
gre            0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa            0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2    -0.6801      0.317     -2.146      0.032        -1.301    -0.059
prestige_3    -1.3387      0.345     -3.882      0.000        -2.015    -0.663
prestige_4    -1.5534      0.417     -3.721      0.000        -2.372    -0.735
==============================================================================
"""

> ### Question 14.  What are the odds ratios of the different features and their 95% confidence intervals?

In [14]:
np.exp(model.params)

Intercept     0.020716
gre           1.002221
gpa           2.180027
prestige_2    0.506548
prestige_3    0.262192
prestige_4    0.211525
dtype: float64

In [15]:
np.exp(model.conf_int(alpha = .05)).\
    rename(columns = {0: '2.5%', 1: '97.5%'})

,2.5%,97.5%
Intercept,0.002207,0.194440
gre,1.000074,1.004372
gpa,1.136120,4.183113
prestige_2,0.272168,0.942767
prestige_3,0.133377,0.515419
prestige_4,0.093329,0.479411


> ### Question 15.  Interpret the odds ratio for `prestige = 2`.

Answer: Students who attended a tier 2 undergraduate school had 0.50 the odds of being admitted to graduate school compared to students who attend a tier 1 undergraduate school.

> ### Question 16.  Interpret the odds ratio of `gpa`.

Answer: For `gpa`, the odds ratio change is about 2, meaning the likelihood of being admitted double as you increase the GPA by 1.

> ### Question 17.  Assuming a student with a GRE of 800 and a GPA of 4.  What is his/her probability of admission  if he/she come from a tier-1, tier-2, tier-3, or tier-4 undergraduate school?

In [16]:
predict_X = pd.DataFrame({'intercept': [1, 1, 1, 1],
    'gre': [800, 800, 800, 800],
    'gpa': [4, 4, 4, 4],
    'prestige_2': [0, 1, 0, 0],
    'prestige_3': [0, 0, 1, 0],
    'prestige_4': [0, 0, 0, 1]})

predict_X

,gpa,gre,intercept,prestige_2,prestige_3,prestige_4
0,4,800,1,0,0,0
1,4,800,1,1,0,0
2,4,800,1,0,1,0
3,4,800,1,0,0,1


In [17]:
model.predict(predict_X)

array([ 0.73403998,  0.58299512,  0.41983282,  0.36860803])

Answer: The tier-1 student has a 73% probability of being admitted; the the lower tiers, the probabilities decrease to 58%, 42%, and 37% respectively.

## Part D. Moving the model from `statsmodels` to `sklearn`

> ### Question 18.  Let's assume we are satisfied with our model.  Remodel it (same features) using `sklearn`.  When creating the logistic regression model with `LogisticRegression(C = 10 ** 2)`.

In [18]:
X = df[ ['gre', 'gpa', 'prestige_2', 'prestige_3', 'prestige_4'] ]
y = df.admit

model = linear_model.LogisticRegression(C = 10 ** 2).fit(X, y)

> ### Question 19.  What are the odds ratios for the different variables and how do they compare with the odds ratios calculated with `statsmodels`?

In [19]:
print np.exp(model.intercept_)
print np.exp(model.coef_)

[ 0.02975414]
[[ 1.00216055  1.96041259  0.53321936  0.28586733  0.20829663]]


Answer: They are close but different nonetheless.  The short answer for why is that the loss function between both implementation differs slightly.

> ### Question 20.  Again, assuming a student with a GRE of 800 and a GPA of 4.  What is his/her probability of admission  if he/she come from a tier-1, tier-2, tier-3, or tier-4 undergraduate school?

In [20]:
predict_X

,gpa,gre,intercept,prestige_2,prestige_3,prestige_4
0,4,800,1,0,0,0
1,4,800,1,1,0,0
2,4,800,1,0,1,0
3,4,800,1,0,0,1


In [21]:
predict_X.drop('intercept', axis = 1, inplace = True)

In [22]:
model.predict_proba(predict_X[ ['gre', 'gpa', 'prestige_2', 'prestige_3', 'prestige_4'] ])

array([[ 0.28814605,  0.71185395],
       [ 0.43153702,  0.56846298],
       [ 0.58608936,  0.41391064],
       [ 0.66024514,  0.33975486]])

Answer: The tier-1 student has a 71% probability of being admitted; the the lower tiers, the probabilities decrease to 57%, 41%, and 34% respectively.  These results are closed to the ones of the `statsmodels`-based model (73%, 58%, 42%, and 37%).